In [1]:
import pandas as pd
df = pd.read_csv(r'D:\charging_data\kor_ev_charging_data_preprocessing.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605417 entries, 0 to 605416
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   loc              605417 non-null  object 
 1   id               605417 non-null  int64  
 2   charging_value   605417 non-null  float64
 3   charging_hour    605417 non-null  int64  
 4   charging_minute  605417 non-null  int64  
 5   date             605417 non-null  object 
 6   start_time       605417 non-null  object 
 7   end_time         605417 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 37.0+ MB
None


In [2]:
# 쓸모없는 열 삭제
df = df.drop('charging_value', axis=1)

df['loc'].value_counts()

loc
대전세종충남    63263
대구        60295
제주        60123
광주전남      55369
경기        55283
인천        44471
부산울산      44210
남서울       38115
서울        37948
강원        31937
경남        30043
경기북부      25431
충북        23673
전북        22532
경북        12724
Name: count, dtype: int64

In [3]:
# 이상치 데이터 삭제

filter = df['loc'].value_counts()

# 10000보다 작은 loc 추출
to_drop = filter[filter < 10000].index

# to_drop에 들어있지 않은 loc만 추출
df = df[~df['loc'].isin(to_drop)]

# 환경부 충전 데이터 삭제 (지역 확인 어려움)
df = df.drop(df[df['loc'] == '환경부'].index)

df['loc'].value_counts()

loc
대전세종충남    63263
대구        60295
제주        60123
광주전남      55369
경기        55283
인천        44471
부산울산      44210
남서울       38115
서울        37948
강원        31937
경남        30043
경기북부      25431
충북        23673
전북        22532
경북        12724
Name: count, dtype: int64

In [4]:
# 충전 시간 (hour) 확인 후 이상치 제거
df['charging_hour'].value_counts()

charging_hour
0    393062
1    187219
2     25136
Name: count, dtype: int64

In [5]:
# 5시간 이상 충전한 데이터 삭제
df = df[df['charging_hour'] <= 2]

df['charging_hour'].value_counts()




charging_hour
0    393062
1    187219
2     25136
Name: count, dtype: int64

In [6]:
df = df[df['charging_minute'] >= 5]

df['charging_minute'].value_counts()

charging_minute
60     14780
40      9007
36      8575
37      8457
39      8430
       ...  
236       58
239       58
232       56
238       52
237       52
Name: count, Length: 175, dtype: int64

In [7]:
df['loc'].value_counts()

loc
대전세종충남    61037
대구        57634
제주        57553
광주전남      53232
경기        52928
인천        42773
부산울산      42401
남서울       36679
서울        36568
강원        30596
경남        28518
경기북부      24268
충북        22846
전북        21561
경북        12202
Name: count, dtype: int64

In [8]:
# 3개월 동안 900 번의 충전 세션이 없는 데이터 삭제 (이상치로 간주, 하루 약 3번 충전)

counts = df['id'].value_counts()

id_keep = counts[counts > 900].index

df = df[df['id'].isin(id_keep)]

df['id'].value_counts()



id
9813     2823
9812     2578
9814     2424
8847     2060
8848     2018
10900    1714
8850     1538
8849     1479
10618    1363
10418    1309
10394    1279
10419    1273
8043     1223
10959    1156
10420    1145
10901    1103
10955    1082
628      1082
631      1065
10958    1007
629       995
11        989
630       987
10417     943
632       912
Name: count, dtype: int64

In [9]:
# 요일을 0-월 1-화 ... 6-일 로 맵핑 (datetime 객체 이용)

df['date'] = pd.to_datetime(df['date'])

df['date'] = df['date'].dt.weekday

df['date'].value_counts()

date
1    5384
0    5382
3    5195
4    5151
2    4978
5    4850
6    4607
Name: count, dtype: int64

In [10]:
# 주말인지 아닌지 여부 데이터 추가

def is_weekend(day):
    if day in [0,1,2,3,4]:
        return 0
    else:
        return 1

df['weekend'] = df['date'].apply(is_weekend)


In [11]:
# 하루를 10분 단위의 144개로 나누어, 충전 시작 시간에 맞게 할당
def time_calculate(time):
    hour, minute = time.split()[1].split(':')
    total_minutes = int(hour) * 60 + int(minute)
    time_interval = total_minutes // 10 
    return time_interval

df['time'] = df['start_time'].apply(time_calculate)

In [13]:
#새로운 df 구성


charging_df = df.iloc[:, [8, 7, 4, 1]]
charging_df = charging_df.drop_duplicates()

# id 별 중복 값이 너무 많은 것인지 id당 3개월에 하루 1세션도 안됨.
charging_df["id"].value_counts()

id
8847     720
8848     696
10618    633
628      608
631      605
8850     601
8849     588
629      587
10419    585
11       566
10418    559
630      554
632      544
10420    534
10959    496
10955    486
10958    483
10417    473
9812     469
9813     436
9814     436
10900    377
10394    364
8043     323
10901    303
Name: count, dtype: int64